This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv  # if missing this module, simply run `pip install python-dotenv`

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

print(API_KEY)

wA3zKgY1jfmFz84pAMb8


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
DB_CODE = "FSE"
DS_CODE = "AFX_X"
r = requests.get(f'https://data.nasdaq.com/api/v3/datasets/{DB_CODE}/{DS_CODE}/data.json?api_key={API_KEY}')

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
import json
dataset = r.json()['dataset_data']
print(type(dataset))

<class 'dict'>


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [6]:
# 1. Get data for the whole year 2017
from copy import deepcopy
data_2017 = deepcopy(dataset)
market_data = data_2017['data']
market_data_2017 = []

for data in market_data:
    year = int(data[0][0:4])
    if year == 2017:
        market_data_2017.append(data)
data_2017['data'] = market_data_2017
print(type(data_2017))

<class 'dict'>


In [9]:
data_2017['column_names']

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

In [38]:
# 3. Calculate highest and lowest opening prices
opening_prices = {}
for data in data_2017['data']:
    date = data[0]
    opening_price = data[1]
    if opening_price:
        opening_prices[date] = opening_price

sorted_opening_prices = sorted(opening_prices.items(), key=lambda x:x[1])
print(f"Lowest Price 2017: {sorted_opening_prices[0][1]}")
print(f"Highest Price 2017: {sorted_opening_prices[-1][1]}")

Lowest Price 2017: 34.0
Highest Price 2017: 53.11


In [40]:
# 4. Largest change in any one day
deltas = {}
for data in data_2017['data']:
    date = data[0]
    high = data[2]
    low = data[3]
    if high and low:
        delta = high - low
        deltas[date] = delta
        
sorted_deltas = sorted(deltas.items(), key=lambda x:x[1])
print(f"Largest change in any one day: {sorted_deltas[-1]}")

Largest change in any one day: ('2017-05-11', 2.8100000000000023)


In [46]:
# 5. Largest Change between any two days
two_day_deltas = {}
for i in range(len(data_2017['data']) - 1):
    date1 = data_2017['data'][i][0]
    date2 = data_2017['data'][i+1][0]
    day1 = data_2017['data'][i][4] # Close price
    day2 = data_2017['data'][i+1][4] # Close price
    two_day_deltas[date2 + " --> " + date1] = day1 - day2

sorted_two_day_deltas = sorted(two_day_deltas.items(), key=lambda x:x[1])
print(f"Largest change in any two days: {sorted_two_day_deltas[-1]}")

Largest change in any two days: ('2017-05-10 --> 2017-05-11', 1.7199999999999989)


In [48]:
# 6. Average daily trading volume
volume = []
for data in data_2017['data']:
    if data[6]:
        volume.append(data[6])
print(f"Average daily trading volume {sum(volume)/len(volume)}")

Average daily trading volume 89124.33725490196


In [53]:
# 7. Median daily trading volume
volume.sort()
if len(volume) % 2 == 1:
    idx = len(volume)//2
    print(f"The median trading volume is: {volume[idx]}")
else:
    idx1 = len(volume)//2
    idx2 = idx1 - 1
    print(f"The median trading volume is: {(volume[idx1]+volume[idx2])//2}")

The median trading volume is: 76286.0
